# December 2017: Advent of Code
## Parts 7-12

## Common imports & library functions

In [108]:
from collections import defaultdict, namedtuple
import doctest
import functools
import heapq
import itertools
import math
import numpy as np
import re

# Cribbed from norvig@
def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(itertools.islice(iterable, n, None), default)

def np_impulse(shape, idx, value, dtype=np.int):
    data = np.zeros(shape, dtype=dtype)
    data[idx] = value
    return data

## Day 7: Recursive Circus

In [ ]:
node_re = re.compile(r"([a-z]+) \(([\-0-9]+)\)(?: -> ([\w, ]+))?")

Node = namedtuple('Node', ['id', 'weight', 'children'])
Tree = namedtuple('Tree', ['root', 'nodes'])

def parse_node(text):
    """
    >>> parse_node("ktlj (57)")
    Node(id='ktlj', weight=57, children=[])
    >>> parse_node("xyz (-7) -> abc, def, ghi")
    Node(id='xyz', weight=-7, children=['abc', 'def', 'ghi'])
    """
    node_id, weight, children = node_re.match(text).groups()
    weight = int(weight)
    children = [c.strip() for c in children.split(',')] if children else []
    return Node(node_id, weight, children)

def parse_tree(text):
    """
    >>> tree = parse_tree("ktlj (57)\\nxyz (-7) -> ktlj")
    >>> tree.root
    'xyz'
    """
    nodes = {}
    for line in text.splitlines():
        line = line.strip()
        if not line: continue
        node = parse_node(line)
        nodes[node.id] = node
    root = get_root(nodes)
    return Tree(root, nodes)


def tree_weight(tree, node_id=None):
    """
    >>> t = Tree('xyz', {'xyz': Node('xyz', 10, ['abc']), 'abc': Node('abc', -2, [])})
    >>> tree_weight(t, 'xyz')
    8
    >>> tree_weight(t, 'abc')
    -2
    >>> tree_weight(t)
    8
    """
    node_id = node_id or tree.root
    node = tree.nodes[node_id]
    return node.weight + sum(tree_weight(tree, c) for c in node.children)

def verify_tree(tree, node_id=None):
    node_id = node_id or tree.root
    node = tree.nodes[node_id]
    if not node.children:
        return node.weight
    ws = [verify_tree(tree, c) for c in node.children]
    if len(set(ws)) != 1:
        bad_idx = list((ws[i] == ws[i-1]) or (ws[i] == ws[i-2]) 
                       for i in range(len(ws))).index(False)
        bad_node = tree.nodes[node.children[bad_idx]]
        bad_weight = ws[bad_idx]
        good_weight = ws[bad_idx - 1]
        correction = (good_weight - bad_weight)
        raise Exception('Failed verification at node <{}> due to child <{}>'.format(node_id, 
                                                                                    bad_node.id),
                        node.children[bad_idx],
                        bad_node.weight + correction)
    else:
        return node.weight + sum(ws)
        

def get_root(nodes):
    """
    >>> get_root({'xyz': Node('xyz', 1, children=['abc']), 'abc': Node('abc', 2, [])})
    'xyz'
    """
    all_nodes = set(nodes)
    child_nodes = set(
        itertools.chain.from_iterable(n.children for n in nodes.values()))
    return (all_nodes - child_nodes).pop()
        
def solve_bottom_program(tree):
    return tree.root

In [ ]:
doctest.testmod(verbose=True)

test_tree = parse_tree("""
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
""")

print('Root is:', test_tree.root)
assert test_tree.root == 'tknk'

try:
    verify_tree(test_tree)
except Exception as e:
    msg, bad_node, corrected_weight = e.args
    assert corrected_weight == 60

In [ ]:
# Final answer
with open('day7.txt') as f:
    tree = parse_tree(f.read())
    print('Part 1: root node is', tree.root)
    try:
        verify_tree(tree)
    except Exception as e:
        print('Part 2: corrected weight is', e.args[2])

## Day 8: I Heard You Like Registers

In [2]:
def Registers(init_values={}):
    return defaultdict(int, init_values)

def print_registers(registers):
    return ' | '.join('{}: {}'.format(*kv) for kv in sorted(registers.items()))
    
def parse_instruction(line):
    """
    >>> parse_instruction('a inc 5 if b == 3')
    ('a', 'inc', 5, 'b', '==', 3)
    """
    var1, op, opval, _, var2, comp, compval = line.split()
    return (var1, op, int(opval), 
            var2, comp, int(compval))

def parse_program(program_text):
    """
    >>> parse_program('a inc 5 if b == 3')
    [('a', 'inc', 5, 'b', '==', 3)]
    >>> parse_program('a inc 5 if b == 3\\nc dec -10 if a >= 1')
    [('a', 'inc', 5, 'b', '==', 3), ('c', 'dec', -10, 'a', '>=', 1)]
    """
    return [parse_instruction(line) for line in program_text.splitlines() if line]

def eval_cond(cond, registers):
    """
    >>> eval_cond(('a', '==', 4), {'a': 4})
    True
    >>> eval_cond(('b', '>', 4), {'b': -3})
    False
    """
    reg, op, val = cond
    return eval('registers["{}"] {} {}'.format(reg, op, val))

def eval_op(op, registers):
    reg, op, val = op
    op = '+=' if op == 'inc' else '-='
    expr = 'registers["{}"] {} {}'.format(reg, op, val)
    eval(compile(expr, '<string>', 'single'))

def run_instruction(instruction, registers):
    """
    >>> registers = Registers({'b': 3})
    >>> print_registers(run_instruction(('a', 'inc', 5, 'b', '==', '3'), registers))
    'a: 5 | b: 3'
    """
    if eval_cond(instruction[3:], registers):
        eval_op(instruction[:3], registers)
    return registers

def run_program(instructions, registers=None):
    """
    >>> registers = Registers({'b': 3})
    >>> print_registers(run_program([('a', 'inc', 5, 'b', '==', '3')], registers))
    'a: 5 | b: 3'
    """
    registers = registers or Registers()
    for instruction in instructions:
        run_instruction(instruction, registers)
    return registers

def solve_max_register_value(program_text):
    instructions = parse_program(program_text)
    registers = Registers()
    historical_max = 0
    for instruction in instructions:
        run_instruction(instruction, registers)
        historical_max = max(historical_max, max(registers.values()))
    return max(registers.values()), historical_max

In [3]:
doctest.testmod(verbose=True)

test_program = """
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10 
"""

current_max, historical_max = solve_max_register_value(test_program)
assert current_max == 1
assert historical_max == 10

Trying:
    eval_cond(('a', '==', 4), {'a': 4})
Expecting:
    True
ok
Trying:
    eval_cond(('b', '>', 4), {'b': -3})
Expecting:
    False
ok
Trying:
    parse_instruction('a inc 5 if b == 3')
Expecting:
    ('a', 'inc', 5, 'b', '==', 3)
ok
Trying:
    parse_program('a inc 5 if b == 3')
Expecting:
    [('a', 'inc', 5, 'b', '==', 3)]
ok
Trying:
    parse_program('a inc 5 if b == 3\nc dec -10 if a >= 1')
Expecting:
    [('a', 'inc', 5, 'b', '==', 3), ('c', 'dec', -10, 'a', '>=', 1)]
ok
Trying:
    registers = Registers({'b': 3})
Expecting nothing
ok
Trying:
    print_registers(run_instruction(('a', 'inc', 5, 'b', '==', '3'), registers))
Expecting:
    'a: 5 | b: 3'
ok
Trying:
    registers = Registers({'b': 3})
Expecting nothing
ok
Trying:
    print_registers(run_program([('a', 'inc', 5, 'b', '==', '3')], registers))
Expecting:
    'a: 5 | b: 3'
ok
7 items had no tests:
    __main__
    __main__.Registers
    __main__.eval_op
    __main__.np_impulse
    __main__.nth
    __main__.print_r

In [4]:
# Final answer
with open('day8.txt') as f:
    current_max, historical_max = solve_max_register_value(f.read())
    print('Part 1: current max value is', current_max)
    print('Part 1: historical max value is', historical_max)

Part 1: current max value is 4647
Part 1: historical max value is 5590


## Day 9: Stream Processing

In [92]:
START_GARBAGE = '<'
END_GARBAGE = '>'
IGNORE_GARBAGE = '!'
START_GROUP = '{'
END_GROUP = '}'
SEPARATOR = ','

def eat_garbage(stream):
    """
    >>> eat_garbage('abc')
    ('abc', 0)
    >>> eat_garbage('<>')
    ('', 0)
    >>> eat_garbage('<abc>123')
    ('123', 3)
    >>> eat_garbage('<!<{!>}>xyxy')
    ('xyxy', 2)
    >>> eat_garbage('<{o"i!a,<{i<a>')
    ('', 10)
    >>> eat_garbage('<!!>')
    ('', 0)
    """
    i = 0
    eaten = 0
    if not stream or stream[i] != START_GARBAGE:
        return (stream, eaten)
    i += 1
    while i < len(stream):
        if stream[i] == IGNORE_GARBAGE:
            i += 2
        elif stream[i] == END_GARBAGE:
            return (stream[i + 1:], eaten)
        else:
            eaten += 1
            i += 1

def eat_separator(stream):
    """
    >>> eat_separator('123,')
    '123,'
    >>> eat_separator(',123,')
    '123,'
    """
    if stream and stream[0] == SEPARATOR:
        return stream[1:]
    else:
        return stream

def eat_group(stream, score=1):
    """
    >>> eat_group('{}')
    ('', 1, 0)
    >>> eat_group('{{},{}}')
    ('', 5, 0)
    >>> eat_group('{{<!!>},{<!!>},{<!!>},{<!!>}}')
    ('', 9, 0)
    >>> eat_group('{{<<<<>},{<!!>},{<<<<>},{<!!>}}')
    ('', 9, 6)
    """
    if not stream or stream[0] != START_GROUP:
        return (stream, 0, 0)
    stream = stream[1:]
    total_score = score
    total_disposed = 0
    while stream:
        # Consume garbage, if any.
        stream, disposed = eat_garbage(stream)
        total_disposed += disposed
        # Consume group, if any.
        stream, group_score, disposed = eat_group(stream, score + 1)
        total_score += group_score
        total_disposed += disposed
        stream = eat_separator(stream)
        if stream[0] == END_GROUP:
            return (stream[1:], total_score, total_disposed)

In [93]:
doctest.testmod(verbose=True)

Trying:
    eat_garbage('abc')
Expecting:
    ('abc', 0)
ok
Trying:
    eat_garbage('<>')
Expecting:
    ('', 0)
ok
Trying:
    eat_garbage('<abc>123')
Expecting:
    ('123', 3)
ok
Trying:
    eat_garbage('<!<{!>}>xyxy')
Expecting:
    ('xyxy', 2)
ok
Trying:
    eat_garbage('<{o"i!a,<{i<a>')
Expecting:
    ('', 10)
ok
Trying:
    eat_garbage('<!!>')
Expecting:
    ('', 0)
ok
Trying:
    eat_group('{}')
Expecting:
    ('', 1, 0)
ok
Trying:
    eat_group('{{},{}}')
Expecting:
    ('', 5, 0)
ok
Trying:
    eat_group('{{<!!>},{<!!>},{<!!>},{<!!>}}')
Expecting:
    ('', 9, 0)
ok
Trying:
    eat_group('{{<<<<>},{<!!>},{<<<<>},{<!!>}}')
Expecting:
    ('', 9, 6)
ok
Trying:
    eat_separator('123,')
Expecting:
    '123,'
ok
Trying:
    eat_separator(',123,')
Expecting:
    '123,'
ok
3 items had no tests:
    __main__
    __main__.np_impulse
    __main__.nth
3 items passed all tests:
   6 tests in __main__.eat_garbage
   4 tests in __main__.eat_group
   2 tests in __main__.eat_separator
12 test

TestResults(failed=0, attempted=12)

In [95]:
# Final answer
with open('day9.txt') as f:
    remainder, score, disposed = eat_group(f.read())
    print('Part 1: group score is', score)
    print('Part 1: disposed of characters', disposed)

Part 1: group score is 10820
Part 1: disposed of characters 5547


## Day 10: Knot Hash

In [152]:
def twist(elements, start, length):
    """
    >>> twist([1, 2, 3, 4, 5], 1, 1)
    [1, 2, 3, 4, 5]
    >>> twist([1, 2, 3, 4, 5], 1, 2)
    [1, 3, 2, 4, 5]
    >>> twist([1, 2, 3, 4, 5], 1, 3)
    [1, 4, 3, 2, 5]
    >>> twist([1, 2, 3, 4, 5], 3, 4)
    [5, 4, 3, 2, 1]
    """
    twisted = elements[:]
    end = (start + length - 1)
    for i in range(length // 2):
        s = (start + i) % len(elements)
        e = (end - i) % len(elements)
        twisted[s] = elements[e]
        twisted[e] = elements[s]
    return twisted

def single_knot_hash(elements, lengths, prev_state=None):
    """
    >>> single_knot_hash([0, 1, 2, 3, 4], [3, 4, 1, 5])
    [3, 4, 2, 1, 0]
    >>> single_knot_hash([0, 1, 2, 3, 4], [3, 4, 1, 5], (0, 0))
    ([3, 4, 2, 1, 0], (4, 4))
    """
    skip, position = prev_state or (0, 0) 
    for length in lengths:
        elements = twist(elements, position, length)
        position = (position + length + skip) % len(elements)
        skip += 1
    if prev_state:
        return (elements, (skip, position))
    else:
        return elements

def ascii_to_bytes(string):
    """
    >>> ascii_to_bytes("1,2,3")
    [49, 44, 50, 44, 51]
    """
    return list(bytes(string, 'ascii'))

def xor_block(block):
    """
    >>> xor_block([65, 27, 9, 1, 4, 3, 40, 50, 91, 7, 6, 0, 2, 5, 68, 22])
    64
    """
    return functools.reduce(lambda x, y: x ^ y, block)

def sparse_to_dense(hash, block_size=16):
    """
    """
    assert (len(hash) % block_size) == 0
    num_blocks = len(hash) // block_size
    dense_hash = []
    for i in range(num_blocks):
        j = i * block_size
        dense_hash.append(xor_block(hash[j:j+block_size]))
    return dense_hash

def dense_to_hex(hash):
    """
    >>> dense_to_hex([64, 7, 255])
    '4007ff'
    """
    return ''.join('%02x' % b for b in hash)

def full_knot_hash(elements, lengths, repetitions=64):
    """
    >>> full_knot_hash(list(range(256)), '')
    'a2582a3a0e66e6e86e3812dcb672a272'
    >>> full_knot_hash(list(range(256)), 'AoC 2017')
    '33efeb34ea91902bb2f59c9920caa6cd'
    >>> full_knot_hash(list(range(256)), '1,2,3')
    '3efbe78a8d82f29979031a4aa0b16a9d'
    >>> full_knot_hash(list(range(256)), '1,2,4')
    '63960835bcdc130f0b66d7ff4f6a5a8e'
    """
    lengths = ascii_to_bytes(lengths) + [17, 31, 73, 47, 23]
    prev_state = (0, 0)
    for _ in range(repetitions):
        elements, prev_state = single_knot_hash(elements, lengths, prev_state)
    return dense_to_hex(sparse_to_dense(elements))

In [153]:
doctest.testmod(verbose=True)

Trying:
    ascii_to_bytes("1,2,3")
Expecting:
    [49, 44, 50, 44, 51]
ok
Trying:
    dense_to_hex([64, 7, 255])
Expecting:
    '4007ff'
ok
Trying:
    full_knot_hash(list(range(256)), '')
Expecting:
    'a2582a3a0e66e6e86e3812dcb672a272'
ok
Trying:
    full_knot_hash(list(range(256)), 'AoC 2017')
Expecting:
    '33efeb34ea91902bb2f59c9920caa6cd'
ok
Trying:
    full_knot_hash(list(range(256)), '1,2,3')
Expecting:
    '3efbe78a8d82f29979031a4aa0b16a9d'
ok
Trying:
    full_knot_hash(list(range(256)), '1,2,4')
Expecting:
    '63960835bcdc130f0b66d7ff4f6a5a8e'
ok
Trying:
    knot_hash(5, [3, 4, 1, 5])
Expecting:
    [3, 4, 2, 1, 0]
ok
Trying:
    single_knot_hash([0, 1, 2, 3, 4], [3, 4, 1, 5])
Expecting:
    [3, 4, 2, 1, 0]
ok
Trying:
    single_knot_hash([0, 1, 2, 3, 4], [3, 4, 1, 5], (0, 0))
Expecting:
    ([3, 4, 2, 1, 0], (4, 4))
ok
Trying:
    twist([1, 2, 3, 4, 5], 1, 1)
Expecting:
    [1, 2, 3, 4, 5]
ok
Trying:
    twist([1, 2, 3, 4, 5], 1, 2)
Expecting:
    [1, 3, 2, 4, 5]
ok
Tryi

TestResults(failed=0, attempted=14)

In [155]:
# Final answer
with open('day10.txt') as f:
    elements = list(range(256))
    raw_lengths = f.read().strip()
    lengths = [int(n) for n in raw_lengths.split(',')]
    hashed = single_knot_hash(elements, lengths)
    print('Part 1: product of first two numbers is', hashed[0] * hashed[1])
    hashed = full_knot_hash(elements, raw_lengths)
    print('Part 2: full hash is', hashed)

Part 1: product of first two numbers is 5577
Part 2: full hash is 44f4befb0f303c0bafd085f97741d51d
